In [10]:
from vinafood_dataset import get_data_loaders
from ResNet import ResNet
from GoogLeNet import GoogLeNet
from pretrain_resnet import PretrainedResnet
import torch
from torch import nn, optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import warnings
warnings.filterwarnings('ignore')

# Settings
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
EPOCHS = 10
BATCH_SIZE = 32
LR = 0.001

# Model
def _make_model(model: str, num_classes: int, lr: float):   # GoogLeNet, ResNet
    if model == "GoogLeNet":
        model = GoogLeNet(n_classes=num_classes).to(device)
    elif model == "ResNet18":
        model = ResNet(num_classes=num_classes).to(device)
    elif model == "ResNet50":
        model = PretrainedResnet().to(device)
    else:
        raise ValueError("Model name không hợp lệ. Chỉ chấp nhận 'GoogLeNet' hoặc 'ResNet'.")
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        model.parameters(),
        lr=LR
    )
    return model, loss_fn, optimizer

# Train Function
def train_model(model, loss_fn, optimizer, loader):
    model.train()
    total_loss = 0.0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        
        # 1. Optimize zero grad
        optimizer.zero_grad()
        
        # 2. Forward Pass
        output_pred = model(images)
        
        # 3. Calculate the loss
        loss = loss_fn(output_pred, labels)
        
        # 4. Perform backpropagation
        loss.backward()
        
        # 5. Step the optimizer
        optimizer.step()
        
        # 6. Update total loss
        total_loss += loss.item()
        
    return total_loss / len(loader)

# Evaluate Function
def evaluate_model(model, loader):
    model.eval()
    y_true, y_pred = [], []
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            output_pred = model(images)
            _, preds = torch.max(output_pred, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    
    return acc, prec, rec, f1

def training_loop(
    epochs, model, model_name, loss_fn,
    optimizer, train_loader, test_loader
):
    for epoch in range(epochs):
        train_loss = train_model(
            model=model, 
            loss_fn=loss_fn,
            optimizer=optimizer,
            loader=train_loader
        )
        acc, prec, rec, f1 = evaluate_model(model=model, loader=test_loader)
        print(f"Epoch [{epoch+1}/{EPOCHS}]")
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1-score: {f1:.4f}")
        print("-" * 50)
    torch.save (model.state_dict(), f"{model_name}.pth")

In [ ]:
if __name__ == "__main__":
    # Data Loader
    train_loader, test_loader, num_classes = get_data_loaders(batch_size=BATCH_SIZE)

    model, loss_fn, optimizer = _make_model("GoogLeNet", num_classes, LR)

    training_loop(
        epochs=EPOCHS,
        model=model,
        model_name="vinamodel",
        loss_fn=loss_fn,
        optimizer=optimizer,
        train_loader=train_loader,
        test_loader=test_loader
    )

/Users/kittnguyen/Downloads/Learning_Documents/Deep_Learning/myvenv/lib/python3.13/site-packages/PIL/Image.py:1039: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/30]
Train Loss: 2.6086
Accuracy: 0.1883 | Precision: 0.1808 | Recall: 0.1918 | F1-score: 0.1433
--------------------------------------------------


/Users/kittnguyen/Downloads/Learning_Documents/Deep_Learning/myvenv/lib/python3.13/site-packages/PIL/Image.py:1039: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [2/30]
Train Loss: 2.3267
Accuracy: 0.2595 | Precision: 0.2468 | Recall: 0.2091 | F1-score: 0.1822
--------------------------------------------------


/Users/kittnguyen/Downloads/Learning_Documents/Deep_Learning/myvenv/lib/python3.13/site-packages/PIL/Image.py:1039: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

Epoch [28/30]
Train Loss: 0.2045
Accuracy: 0.5245 | Precision: 0.5533 | Recall: 0.5289 | F1-score: 0.5162
--------------------------------------------------


/Users/kittnguyen/Downloads/Learning_Documents/Deep_Learning/myvenv/lib/python3.13/site-packages/PIL/Image.py:1039: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [29/30]
Train Loss: 0.2161
Accuracy: 0.5663 | Precision: 0.5866 | Recall: 0.5567 | F1-score: 0.5550
--------------------------------------------------


/Users/kittnguyen/Downloads/Learning_Documents/Deep_Learning/myvenv/lib/python3.13/site-packages/PIL/Image.py:1039: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [30/30]
Train Loss: 0.1978
Accuracy: 0.5576 | Precision: 0.5942 | Recall: 0.5482 | F1-score: 0.5515
--------------------------------------------------
Training done, model saved!


In [8]:
if __name__ == "__main__":
    # Data Loader
    train_loader, test_loader, num_classes = get_data_loaders(batch_size=BATCH_SIZE)

    model, loss_fn, optimizer = _make_model("ResNet", num_classes, LR)

    training_loop(
        epochs=EPOCHS,
        model=model,
        model_name="resnet",
        loss_fn=loss_fn,
        optimizer=optimizer,
        train_loader=train_loader,
        test_loader=test_loader
    )

Loaded dataset: 21 classes, 10044 train images, 6682 test images
Epoch [1/10]
Train Loss: 2.6195
Accuracy: 0.2287 | Precision: 0.2670 | Recall: 0.2197 | F1-score: 0.2000
--------------------------------------------------
Epoch [2/10]
Train Loss: 2.3494
Accuracy: 0.2415 | Precision: 0.2724 | Recall: 0.2362 | F1-score: 0.2003
--------------------------------------------------
Epoch [3/10]
Train Loss: 2.1614
Accuracy: 0.3041 | Precision: 0.3421 | Recall: 0.2874 | F1-score: 0.2601
--------------------------------------------------
Epoch [4/10]
Train Loss: 2.0263
Accuracy: 0.3212 | Precision: 0.3485 | Recall: 0.3113 | F1-score: 0.2936
--------------------------------------------------
Epoch [5/10]
Train Loss: 1.8983
Accuracy: 0.3707 | Precision: 0.3715 | Recall: 0.3586 | F1-score: 0.3320
--------------------------------------------------
Epoch [6/10]
Train Loss: 1.7624
Accuracy: 0.3768 | Precision: 0.4065 | Recall: 0.3659 | F1-score: 0.3413
--------------------------------------------------

In [11]:
if __name__ == "__main__":
    # Data Loader
    train_loader, test_loader, num_classes = get_data_loaders(batch_size=BATCH_SIZE)

    model, loss_fn, optimizer = _make_model("ResNet50", num_classes, LR)

    training_loop(
        epochs=EPOCHS,
        model=model,
        model_name="resnet50",
        loss_fn=loss_fn,
        optimizer=optimizer,
        train_loader=train_loader,
        test_loader=test_loader
    )

Loaded dataset: 21 classes, 10044 train images, 6682 test images
Epoch [1/10]
Train Loss: 1.2197
Accuracy: 0.7562 | Precision: 0.7844 | Recall: 0.7549 | F1-score: 0.7543
--------------------------------------------------
Epoch [2/10]
Train Loss: 0.4121
Accuracy: 0.7472 | Precision: 0.7842 | Recall: 0.7495 | F1-score: 0.7518
--------------------------------------------------
Epoch [3/10]
Train Loss: 0.2067
Accuracy: 0.8301 | Precision: 0.8414 | Recall: 0.8267 | F1-score: 0.8294
--------------------------------------------------
Epoch [4/10]
Train Loss: 0.1536
Accuracy: 0.8191 | Precision: 0.8213 | Recall: 0.8268 | F1-score: 0.8185
--------------------------------------------------
Epoch [5/10]
Train Loss: 0.1217
Accuracy: 0.7879 | Precision: 0.8074 | Recall: 0.7834 | F1-score: 0.7835
--------------------------------------------------
Epoch [6/10]
Train Loss: 0.1013
Accuracy: 0.8535 | Precision: 0.8591 | Recall: 0.8507 | F1-score: 0.8521
--------------------------------------------------